In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv1D, LSTM, Dense

# Descargar y cargar el archivo CSV con los datos
data_url = 'https://github.com/watorres/ProyectoDeepLearning/raw/main/device_gnss.csv.zip'
data = pd.read_csv(data_url, compression='zip')

# Preprocesar los datos
data = data[['TimeNanos', 'RawPseudorangeMeters', 'SvPositionXEcefMeters', 'SvPositionYEcefMeters', 'SvPositionZEcefMeters']]
data = data.dropna()
X = data[['TimeNanos', 'RawPseudorangeMeters']]
y = data[['SvPositionXEcefMeters', 'SvPositionYEcefMeters', 'SvPositionZEcefMeters']]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)
X_train_reshaped = X_train_scaled.reshape(X_train_scaled.shape[0], X_train_scaled.shape[1], 1)
X_test_reshaped = X_test_scaled.reshape(X_test_scaled.shape[0], X_test_scaled.shape[1], 1)

# Implementación personalizada de la capa de atención
class Attention(tf.keras.layers.Layer):
    def __init__(self):
        super(Attention, self).__init__()

    def build(self, input_shape):
        self.W = self.add_weight(shape=(input_shape[-1], 1), initializer='normal', trainable=True)
        super(Attention, self).build(input_shape)

    def call(self, inputs):
        score = tf.matmul(inputs, self.W)
        attention_weights = tf.nn.softmax(score, axis=1)
        weighted_sum = tf.reduce_sum(inputs * attention_weights, axis=1)
        return weighted_sum

# Definir y entrenar el modelo
input_layer = Input(shape=(X_train_reshaped.shape[1], X_train_reshaped.shape[2]))
conv1d_layer = Conv1D(64, 1, activation='relu')(input_layer)
lstm_layer = LSTM(64, return_sequences=True)(conv1d_layer)
attention_layer = Attention()(lstm_layer)
output_layer = Dense(3)(attention_layer)
model = Model(inputs=input_layer, outputs=output_layer)
model.compile(optimizer='adam', loss='mse')
model.fit(X_train_reshaped, y_train, epochs=10, batch_size=32, validation_data=(X_test_reshaped, y_test))

# Calcular el error para los percentiles 50 y 95
y_pred = model.predict(X_test_reshaped)
errors = np.sqrt(np.sum((y_pred - y_test.values)**2, axis=1))
percentile_50 = np.percentile(errors, 50)
percentile_95 = np.percentile(errors, 95)

print("Error Percentile 50:", percentile_50)
print("Error Percentile 95:", percentile_95)

Epoch 1/10
1341/1341 [==============================] - 23s 11ms/step - loss: 256431072739328.0000 - val_loss: 256320930316288.0000
Epoch 2/10
1341/1341 [==============================] - 6s 5ms/step - loss: 256428707151872.0000 - val_loss: 256319034490880.0000
Epoch 3/10
1341/1341 [==============================] - 7s 5ms/step - loss: 256427062984704.0000 - val_loss: 256317524541440.0000
Epoch 4/10
1341/1341 [==============================] - 6s 5ms/step - loss: 256425334931456.0000 - val_loss: 256315863597056.0000
Epoch 5/10
1341/1341 [==============================] - 7s 5ms/step - loss: 256423506214912.0000 - val_loss: 256313934217216.0000
Epoch 6/10
1341/1341 [==============================] - 6s 4ms/step - loss: 256421945933824.0000 - val_loss: 256312139055104.0000
Epoch 7/10
1341/1341 [==============================] - 7s 5ms/step - loss: 256420352098304.0000 - val_loss: 256310427779072.0000
Epoch 8/10
1341/1341 [==============================] - 6s 5ms/step - loss: 256418489827


Estos resultados indican que el modelo ha sido entrenado durante 10 épocas. Durante el entrenamiento, se muestra el valor de la función de pérdida (loss) tanto para el conjunto de entrenamiento como para el conjunto de validación (val_loss).

En cada época, se muestra el valor de la función de pérdida tanto para el conjunto de entrenamiento como para el conjunto de validación. En este caso, los valores de pérdida son bastante altos, lo que indica que el modelo no está aprendiendo correctamente los patrones en los datos.

Después del entrenamiento, se calculan los errores para los percentiles 50 y 95 en el conjunto de prueba. El error del percentil 50 (Error Percentile 50) es de aproximadamente 26,714,063.07, lo que significa que el 50% de los errores se encuentran por debajo de este valor. El error del percentil 95 (Error Percentile 95) es de aproximadamente 29,601,549.57, lo que significa que el 95% de los errores se encuentran por debajo de este valor.

En resumen, los resultados indican que el modelo no está obteniendo un buen desempeño en la tarea de predicción, ya que la función de pérdida es alta y los errores son significativos.